## Pegando dados de preços do mercado livre

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

In [2]:
# Seleciono plavras chave para procurar no site do mercado livre
search_commands = ['celular', 'tv', 'geladeira', 'microondas', 'fogao', 'maquiagem', 'iphone']

In [3]:
%%time
data = pd.DataFrame(index=range(0,60*len(search_commands)), columns=['product_name', 'product_price', 'product_search', 'search_date'])
n=0
errors={}
for search in search_commands:
    
    url = 'https://lista.mercadolivre.com.br/{}#D[A:{}]'.format(search, search)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser') 
    items = soup.find_all('li', class_='ui-search-layout__item')
    
    # Salvo os dados de cada item do site em um dataframe
    for item in items:
        try:
            try:
                name = item.find('h2', class_="ui-search-item__title ui-search-item__group__element").text
            except:
                name = item.find('h2', class_="ui-search-item__title").text
            price = item.find('span', class_="price-tag-fraction").text.replace('.', '')

            data.loc[n, 'product_name'] = name
            data.loc[n, 'product_price'] = price
            data.loc[n, 'product_search'] = search
            data.loc[n, 'search_date'] = datetime.today()
            n+=1
        except:
            errors.update({name,price,search})
            pass

Wall time: 11.4 s


In [4]:
# Como a base de dados fica:
data.dropna()

,product_name,product_price,product_search,search_date
0,Samsung Galaxy S10+ Dual SIM 128 GB branco-pri...,3750,celular,2020-11-11 17:53:43.068753
1,Samsung Galaxy Note10 Dual SIM 256 GB Aura bla...,2969,celular,2020-11-11 17:53:43.069753
2,Samsung Galaxy A31 Dual SIM 128 GB prism crush...,2399,celular,2020-11-11 17:53:43.070752
3,Samsung Galaxy A21s Dual SIM 64 GB branco 4 GB...,1999,celular,2020-11-11 17:53:43.071751
4,Samsung Galaxy A01 Core Dual SIM 32 GB azul 2 ...,689,celular,2020-11-11 17:53:43.072751
...,...,...,...,...
341,iPhone 12 64 GB preto,7699,iphone,2020-11-11 17:53:51.956721
342,iPhone 7 128 GB prata,1900,iphone,2020-11-11 17:53:51.957720
343,iPhone SE 32 GB ouro rosa,1299,iphone,2020-11-11 17:53:51.958720
344,iPhone 11 128 GB (Product)Red,5999,iphone,2020-11-11 17:53:51.961719


In [5]:
# Nenhum erro foi apresentado
errors

{}

### Curiosidade

In [6]:
# Ticket médio da busca por 'celular'
data[data['product_search']=='celular']['product_price'].astype('float').mean()

1556.5

In [7]:
# Ticket médio da busca por 'iphone'
data[data['product_search']=='iphone']['product_price'].astype('float').mean()

3637.14